# ทดสอบการสร้างแพ็คเกจ และการรัน Unit Test เกือบสมบูรณ์ (ยกเว้น xfail และ skipif กับ ภาพที่ไม่เหมือน baseline ไม่ถูกสร้างขึ้นมาให้ดู)

## ถ้าทำตามขั้นตอนของคอร์ส developing packages จะไม่มีปัญหาในการ import package ทั้งหมด แต่ต้องไปแก้เพิ่มในส่วนที่เป็น test module ให้มีคำนำหน้า (ในที่นี้ใช้ชื่อ uregress) module data, features, models, visuaization

สร้างโฟลเดอร์ mypackages/uregress


In [ ]:
!mkdir /content/mypackages/uregress

In [40]:
!mkdir /content/mypackages/uregress/data
!mkdir /content/mypackages/uregress/data/clean
!mkdir /content/mypackages/uregress/data/raw

In [1]:
import os
os.chdir("/content/mypackages/uregress")
!pip install -e .

Obtaining file:///content/mypackages/uregress
  Found existing installation: uregress 0.1.0
    Can't uninstall 'uregress'. No files were found to uninstall.
  Running setup.py develop for uregress


In [ ]:
%%file /content/mypackages/uregress/setup.py
from setuptools import setup, find_packages

# Add install requirements
setup(
    author="<your-name>",
    description="A practiced package",
    name="uregress",
    packages=find_packages(include=["uregress", "uregress.*"]),
    version="0.1.0",
    install_requires=["jupyter>=1.0.0",
                        "matplotlib>=3.1.1",
                        "numpy>=1.17.3",
                        "pytest>=5.2.2",
                        "pytest-mpl>=0.10",
                        "pytest-mock>=1.11.2",
                        "scipy>=1.3.1",
                        ],
    python_requires=">=3.5",
)

Writing /content/mypackages/uregress/setup.py


สร้างโฟลเดอร์ mypackages/uregress/uregress

In [ ]:
%%file /content/mypackages/uregress/uregress/__init__.py

__doc__ = """
impyrial
========
A package for converting between imperial measurments of length and weight.
"""

__author__ = """Practice Man"""
__version__ = "0.1.0"

from . import data, features, models, plots  # noqa : F401


Writing /content/mypackages/uregress/uregress/__init__.py


*   สร้างโฟลเดอร์ /content/mypackages/uregress/uregress/data
*   สร้างโฟลเดอร์ /content/mypackages/uregress/uregress/features
*   สร้างโฟลเดอร์ /content/mypackages/uregress/uregress/models
*   สร้างโฟลเดอร์ /content/mypackages/uregress/uregress/plots



In [ ]:
!mkdir /content/mypackages/uregress/uregress/data
!mkdir /content/mypackages/uregress/uregress/features
!mkdir /content/mypackages/uregress/uregress/models
!mkdir /content/mypackages/uregress/uregress/visualization

In [ ]:
%%file /content/mypackages/uregress/uregress/data/__init__.py
from .preprocessing_helpers import *

Writing /content/mypackages/uregress/uregress/data/__init__.py


In [ ]:
%%file /content/mypackages/uregress/uregress/data/preprocessing_helpers.py
def convert_to_int(integer_string_with_commas):
    comma_separated_parts = integer_string_with_commas.split(",")
    for i in range(len(comma_separated_parts)):
        if len(comma_separated_parts[i]) > 3:
            return None
        if i != 0 and len(comma_separated_parts[i]) != 3:
            return None
    integer_string_without_commas = "".join(comma_separated_parts)
    try:
        return int(integer_string_without_commas)
    except ValueError:
        return None


def row_to_list(row):
    row = row.rstrip("\n")
    separated_entries = row.split("\t")
    if len(separated_entries) == 2 and "" not in separated_entries:
        return separated_entries
    return None


def preprocess(raw_data_file_path, clean_data_file_path):
    with open(raw_data_file_path, "r") as input_file:
        rows = input_file.readlines()
    with open(clean_data_file_path, "w") as output_file:
        for row in rows:
            row_as_list = row_to_list(row)
            if row_as_list is None:
                continue
            area = convert_to_int(row_as_list[0])
            price = convert_to_int(row_as_list[1])
            if area is None or price is None:
                continue
            output_file.write("{0}\t{1}\n".format(area, price))


Writing /content/mypackages/uregress/uregress/data/preprocessing_helpers.py


In [ ]:
%%file /content/mypackages/uregress/uregress/features/__init__.py
from .as_numpy import *

Writing /content/mypackages/uregress/uregress/features/__init__.py


In [ ]:
%%file /content/mypackages/uregress/uregress/features/as_numpy.py
import numpy as np


def get_data_as_numpy_array(clean_data_file_path, num_columns):
    result = np.empty((0, num_columns))
    with open(clean_data_file_path, "r") as f:
        rows = f.readlines()
        for row_num in range(len(rows)):
            try:
                row = np.array([rows[row_num].rstrip("\n").split("\t")], dtype=float)
            except ValueError:
                raise ValueError("Line {0} of {1} is badly formatted".format(row_num + 1, clean_data_file_path))
            else:
                if row.shape != (1, num_columns):
                    raise ValueError("Line {0} of {1} does not have {2} columns".format(row_num + 1,
                                                                                        clean_data_file_path,
                                                                                        num_columns
                                                                                        )
                                     )
            result = np.append(result, row, axis=0)
    return result


Writing /content/mypackages/uregress/uregress/features/as_numpy.py


In [ ]:
%%file /content/mypackages/uregress/uregress/models/__init__.py
from .train import *

Writing /content/mypackages/uregress/uregress/models/__init__.py


In [ ]:
%%file /content/mypackages/uregress/uregress/models/train.py
import numpy as np
from scipy.stats import linregress


def split_into_training_and_testing_sets(data_array):
    dim = data_array.ndim
    if dim != 2:
        raise ValueError("Argument data_array must be two dimensional. Got {0} dimensional array instead!".format(dim))
    num_rows = data_array.shape[0]
    if num_rows < 2:
        raise ValueError("Argument data_array must have at least 2 rows, it actually has just {0}".format(num_rows))
    num_training = int(0.75 * data_array.shape[0])
    permuted_indices = np.random.permutation(data_array.shape[0])
    return data_array[permuted_indices[:num_training], :], data_array[permuted_indices[num_training:], :]


def model_test(testing_set, slope, intercept):
    dim = testing_set.ndim
    if dim != 2:
        raise ValueError("Argument testing_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_cols = testing_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument testing_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    actual_price = testing_set[:, 1]
    predicted_price = slope*testing_set[:, 0] + intercept
    residual_sum_of_squares = np.sum(np.square(predicted_price - actual_price)) / testing_set.shape[0]
    return 1 - residual_sum_of_squares / np.var(actual_price)


def train_model(training_set):
    dim = training_set.ndim
    if dim != 2:
        raise ValueError("Argument training_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_rows = training_set.shape[0]
    if num_rows < 2:
        raise ValueError("Argument training_set must have at least 2 rows for linear regression to work, "
                         "it actually has just {0}".format(num_rows)
                         )
    num_cols = training_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument training_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    slope, intercept, _, _, _ = linregress(training_set[:, 0], training_set[:, 1])
    return slope, intercept



Writing /content/mypackages/uregress/uregress/models/train.py


In [ ]:
%%file /content/mypackages/uregress/uregress/plots/__init__.py
from .plots import *

Writing /content/mypackages/uregress/uregress/plots/__init__.py


In [67]:
%%file /content/mypackages/uregress/uregress/visualization/plots.py
import matplotlib.pyplot as plt
import numpy as np


def get_plot_for_best_fit_line(slope, intercept, x_array, y_array, title):
    x_array_ndim = x_array.ndim
    if not x_array_ndim == 1:
        raise ValueError("Argument x_array should be 1 dimensional. "
                         "It actually is {0} dimensional".format(x_array_ndim)
                         )
    y_array_ndim = y_array.ndim
    if not y_array_ndim == 1:
        raise ValueError("Argument y_array should be 1 dimensional. "
                         "It actually is {0} dimensional".format(y_array_ndim)
                         )
    x_array_length = x_array.shape[0]
    y_array_length = y_array.shape[0]
    if x_array_length != y_array_length:
        raise RuntimeError("Arguments x_array and y_array should have same length. "
                           "But x_array has length {0} and y_array has length {1}".format(x_array_length,
                                                                                          y_array_length
                                                                                          )
                           )
    fig, ax = plt.subplots()
    ax.plot(x_array, y_array, ".")
    ax.plot([0, np.max(x_array)], [intercept, slope * np.max(x_array) + intercept], "-")
    ax.set(xlabel="area (square feet)", ylabel="price (dollars)", title=title)
    return fig


Overwriting /content/mypackages/uregress/uregress/visualization/plots.py


In [ ]:
!mkdir /content/mypackages/uregress/tests

In [ ]:
!mkdir /content/mypackages/uregress/tests/data
!mkdir /content/mypackages/uregress/tests/features
!mkdir /content/mypackages/uregress/tests/models
!mkdir /content/mypackages/uregress/tests/visualization
!mkdir /content/mypackages/uregress/tests/visualization/baseline

In [ ]:
%%file /content/mypackages/uregress/tests/__init__.py

"""Unit tests for uregress."""

Overwriting /content/mypackages/uregress/tests/__init__.py


In [ ]:
%%file /content/mypackages/uregress/tests/data/__init__.py

"""Unit tests for preprocessing."""

Writing /content/mypackages/uregress/tests/data/__init__.py


In [ ]:
%%file /content/mypackages/uregress/tests/features/__init__.py

"""Unit tests for as_numpy."""

Writing /content/mypackages/uregress/tests/features/__init__.py


In [ ]:
%%file /content/mypackages/uregress/tests/models/__init__.py

"""Unit tests for train."""

Writing /content/mypackages/uregress/tests/models/__init__.py


In [ ]:
%%file /content/mypackages/uregress/tests/plots/__init__.py

"""Unit tests for plots."""

Writing /content/mypackages/uregress/tests/plots/__init__.py


In [53]:
%%file /content/mypackages/uregress/tests/data/test_preprocessing_helpers.py

from unittest.mock import call

import pytest

from uregress.data.preprocessing_helpers import convert_to_int, row_to_list, preprocess


@pytest.fixture
def raw_and_clean_data_file(tmpdir):
    raw_path = tmpdir.join("raw.txt")
    clean_path = tmpdir.join("clean.txt")
    with open(raw_path, "w") as f:
        f.write("1,801\t201,411\n"
                "1,767565,112\n"
                "2,002\t333,209\n"
                "1990\t782,911\n"
                "1,285\t389129\n"
                )
    return raw_path, clean_path


def row_to_list_bug_free(row):
    return_values = {"1,801\t201,411\n": ["1,801", "201,411"],
                     "1,767565,112\n": None,
                     "2,002\t333,209\n": ["2,002", "333,209"],
                     "1990\t782,911\n": ["1990", "782,911"],
                     "1,285\t389129\n": ["1,285", "389129"],
                     }
    return return_values[row]

# Define a function convert_to_int_bug_free() which takes one argument called 
# comma_separated_integer_string.
def convert_to_int_bug_free(comma_separated_integer_string): 
    return_values = {"1,801": 1801, # Define a function convert_to_int_bug_free() 
                     "201,411": 201411, # which takes one argument called comma_separated_integer_string.
                     "2,002": 2002,
                     "333,209": 333209,
                     "1990": None,
                     "782,911": 782911,
                     "1,285": 1285,
                     "389129": None,
                     }
    # Return the correct return value by looking up the dictionary return_values 
    # for the key comma_separated_integer_string
    return return_values[comma_separated_integer_string]


class TestConvertToInt(object):
    def test_with_no_comma(self):
        test_argument = "756"
        expected = 756
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: 756, Actual: {0}".format(actual)

    def test_with_one_comma(self):
        test_argument = "2,081"
        expected = 2081
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: 2081, Actual: {0}".format(actual)

    def test_with_two_commas(self):
        test_argument = "1,034,891"
        expected = 1034891
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: 1034891, Actual: {0}".format(actual)

    def test_on_string_with_incorrectly_placed_comma(self):
        test_argument = "12,72,891"
        expected = None
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: None, Actual: {0}".format(actual)

    def test_on_string_with_missing_comma(self):
        test_argument = "178100,301"
        expected = None
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: None, Actual: {0}".format(actual)

    def test_on_float_valued_string(self):
        test_argument = "6.9"
        expected = None
        actual = convert_to_int(test_argument)
        assert actual == expected, "Expected: None, Actual: {0}".format(actual)


class TestRowToList(object):
    def test_on_no_tab_no_missing_value(self):    # (0, 0) boundary value
        actual = row_to_list("123\n")
        assert actual is None, "Expected: None, Actual: {0}".format(actual)

    def test_on_two_tabs_no_missing_value(self):    # (2, 0) boundary value
        actual = row_to_list("123\t4,567\t89\n")
        assert actual is None, "Expected: None, Actual: {0}".format(actual)

    def test_on_one_tab_with_missing_value(self):    # (1, 1) boundary value
        actual = row_to_list("\t4,567\n")
        assert actual is None, "Expected: None, Actual: {0}".format(actual)

    def test_on_no_tab_with_missing_value(self):    # (0, 1) case
        actual = row_to_list("\n")
        assert actual is None, "Expected: None, Actual: {0}".format(actual)

    def test_on_two_tabs_with_missing_value(self):    # (0, 1) case
        actual = row_to_list("123\t\t89\n")
        assert actual is None, "Expected: None, Actual: {0}".format(actual)

    def test_on_normal_argument_1(self):
        actual = row_to_list("123\t4,567\n")
        expected = ["123", "4,567"]
        assert actual == expected, "Expected: {0}, Actual: {1}".format(expected, actual)

    def test_on_normal_argument_2(self):
        actual = row_to_list("1,059\t186,606\n")
        expected = ["1,059", "186,606"]
        assert actual == expected, "Expected: {0}, Actual: {1}".format(expected, actual)


class TestPreprocess(object):
    # Add the correct argument "mocker" to use the mocking fixture in this test
    def test_on_raw_data(self, raw_and_clean_data_file, mocker):
        raw_path, clean_path = raw_and_clean_data_file
        row_to_list_mock = mocker.patch("uregress.data.preprocessing_helpers.row_to_list", side_effect=row_to_list_bug_free)
        # Replace the dependency "uregress.data.preprocessing_helpers.convert_to_int" with the bug-free version
        # convert_to_int_bug_free() by using the correct method and side effect.
        convert_to_int_mock = mocker.patch("uregress.data.preprocessing_helpers.convert_to_int",
                                           side_effect=convert_to_int_bug_free
                                           )
        preprocess(raw_path, clean_path)
        # Use the correct attribute which returns the list of calls to the mock, 
        # and check if the mock was called with this sequence of arguments.
        assert row_to_list_mock.call_args_list == [call("1,801\t201,411\n"),
                                                   call("1,767565,112\n"),
                                                   call("2,002\t333,209\n"),
                                                   call("1990\t782,911\n"),
                                                   call("1,285\t389129\n")
                                                   ]
        assert convert_to_int_mock.call_args_list == [call("1,801"), call("201,411"), call("2,002"), call("333,209"),
                                                      call("1990"),  call("782,911"), call("1,285"), call("389129")
                                                      ]
        with open(clean_path, "r") as f:
            lines = f.readlines()
        first_line = lines[0]
        assert first_line == "1801\t201411\n"
        second_line = lines[1]
        assert second_line == "2002\t333209\n"



Overwriting /content/mypackages/uregress/tests/data/test_preprocessing_helpers.py


In [44]:
%%file /content/mypackages/uregress/tests/features/test_as_numpy.py
import os
import numpy as np
import pytest

from uregress.features.as_numpy import get_data_as_numpy_array

# Add the correct decorator that would turn clean_data_file() into a fixture.
@pytest.fixture
def clean_data_file():
    file_path = "clean.txt"
    with open(file_path, "w") as f:
        f.write("201\t305671\n7892\t298140\n501\t738293\n")
    yield file_path
    os.remove(file_path)


# @pytest.fixture
# def empty_file():
#     file_path = "empty.txt" # Assign the file path "empty.txt" to the variable
#     open(file_path, "w").close()
#     yield file_path # Yield the variable file_path
#     os.remove(file_path) # Remove the file in the teardown

### Improved version ###

@pytest.fixture
# Add the correct argument so that this fixture can chain with the tmpdir fixture
def empty_file(tmpdir):
    # Use the appropriate method to create an empty file in the temporary directory
    file_path = tmpdir.join("empty.txt")
    open(file_path, "w").close()
    yield file_path


class TestGetDataAsNumpyArray(object):
    def test_on_clean_file(self, clean_data_file): # Pass an argument to the test test_on_clean_file() so that it uses the fixture.
        expected = np.array([[201.0, 305671.0], [7892.0, 298140.0], [501.0, 738293.0]])
        actual = get_data_as_numpy_array(clean_data_file, 2) # Pass the clean data file path yielded by the fixture as the first argument to the function get_data_as_numpy_array()
        assert actual == pytest.approx(expected), "Expected: {0}, Actual: {1}".format(expected, actual)

    def test_on_empty_file(self, empty_file):
        expected = np.empty((0, 2))
        actual = get_data_as_numpy_array(empty_file, 2)
        assert actual == pytest.approx(expected), "Expected: {0}, Actual: {1}".format(expected, actual)


Overwriting /content/mypackages/uregress/tests/features/test_as_numpy.py


In [57]:
%%file /content/mypackages/uregress/tests/models/test_train.py
from math import cos, pi, sin
import numpy as np
import pytest

from uregress.models.train import split_into_training_and_testing_sets, train_model, model_test


class TestSplitIntoTrainingAndTestingSets(object):
    def test_on_six_rows(self):
        test_argument = np.array([[2081.0, 314942.0],
                               [1059.0, 186606.0],
                               [1148.0, 206186.0],
                               [1506.0, 248419.0],
                               [1210.0, 214114.0],
                               [1697.0, 277794.0],
                               ]
                              )
        expected_length_training_set = 4
        expected_length_testing_set = 2
        actual = split_into_training_and_testing_sets(test_argument)
        assert actual[0].shape[0] == expected_length_training_set, \
               "The actual number of rows in the training array is not 4"
        assert actual[1].shape[0] == expected_length_testing_set, \
               "The actual number of rows in the testing array is not 2"

    def test_on_one_row(self):
        test_argument = np.array([[1382.0, 390167.0]])
        with pytest.raises(ValueError) as exc_info:
            split_into_training_and_testing_sets(test_argument)
        expected_error_msg = "Argument data_array must have at least 2 rows, it actually has just 1"
        assert exc_info.match(expected_error_msg)

    def test_on_one_dimensional_array(self):
        test_argument = np.array([1382.0, 390167.0])
        with pytest.raises(ValueError) as exc_info:
            split_into_training_and_testing_sets(test_argument)
        expected_error_msg = "Argument data_array must be two dimensional. Got 1 dimensional array instead!"
        assert exc_info.match(expected_error_msg)


class TestTrainModel(object):
    def test_on_linear_data(self):
        test_argument = np.array([[1.0, 3.0], [2.0, 5.0], [3.0, 7.0]])
        expected_slope = 2.0
        expected_intercept = 1.0
        actual_slope, actual_intercept = train_model(test_argument)
        slope_message = ("train_model({0}) should return slope {1}, "
                         "but it actually returned slope {2}".format(test_argument, expected_slope, actual_slope)
                         )
        intercept_message = ("train_model({0}) should return intercept {1}, "
                             "but it actually returned intercept {2}".format(test_argument,
                                                                             expected_intercept,
                                                                             actual_intercept
                                                                             )
                             )
        assert actual_slope == pytest.approx(expected_slope), slope_message
        assert actual_intercept == pytest.approx(expected_intercept), intercept_message

    def test_on_positively_correlated_data(self):
        test_argument = np.array([[1.0, 4.0], [2.0, 4.0],
                                  [3.0, 9.0], [4.0, 10.0],
                                  [5.0, 7.0], [6.0, 13.0],
                                  ]
                                 )
        actual_slope, actual_intercept = train_model(test_argument)
        assert actual_slope > 0, "Expected slope: > 0, Actual slope: {0}".format(actual_slope)


class TestModelTest(object):
    def test_on_perfect_fit(self):
        # Assign the variable test_argument to a NumPy array holding the perfectly linear testing data
        test_argument = np.array([[1.0, 3.0], [2.0, 5.0], [3.0, 7.0]])
        # Assign the variable expected to the expected value of r-squared in the special case of a perfect fit.
        expected = 1.0
        # Fill in with the model's slope and intercept that matches the testing set.
        actual = model_test(test_argument, 2.0, 1.0)
        message = "model_test({0}) should return {1}, but it actually returned {2}".format(test_argument,
                                                                                           expected,
                                                                                           actual
                                                                                           )
        # Remembering that actual is a float, complete the assert statement to check if actual returned by 
        # model_test() is equal to the expected return value expected.
        assert actual == pytest.approx(expected), message

    def test_on_circular_data(self):
        theta = pi / 4.0
        # Assign test_argument to a  NumPy array holding the circular testing 
        # data shown in the /content/mypackages/uregress/data/circula_data.png, starting with (1.0, 0.0) and moving anticlockwise.
        test_argument = np.array([[0.0, 1.0],
                                  [cos(theta), sin(theta)],
                                  [1.0, 0.0],
                                  [cos(3 * theta), sin(3 * theta)],
                                  [0.0, -1.0],
                                  [cos(5 * theta), sin(5 * theta)],
                                  [-1.0, 0.0],
                                  [cos(7 * theta), sin(7 * theta)]
                                  ]
                                 )
        # Fill in with the slope and intercept of the straight line shown in the plot.
        actual = model_test(test_argument, 0.0, 0.0)
        # Remembering that model_test() returns a float, complete the assert 
        # statement to check if model_test() returns the expected value of r-squared in this special case.
        assert actual == pytest.approx(0.0)

    def test_on_one_dimensional_array(self):
        test_argument = np.array([1.0, 2.0, 3.0, 4.0])
        with pytest.raises(ValueError) as exc_info:
            model_test(test_argument, 1.0, 1.0)
        expected_error_msg = "Argument testing_set must be two dimensional. Got 1 dimensional array instead!"
        assert exc_info.match(expected_error_msg)





Overwriting /content/mypackages/uregress/tests/models/test_train.py


In [ ]:
%%file /content/mypackages/uregress/tests/visualization/test_plots.py
import pytest
import numpy as np

from uregress.visualization.plots import get_plot_for_best_fit_line


class TestGetPlotForBestFitLine(object):
    @pytest.mark.mpl_image_compare          # Under the hood baseline generation and comparison
    def test_plot_for_linear_data(self):
        slope = 2.0
        intercept = 1.0
        x_array = np.array([1.0, 2.0, 3.0])
        y_array = np.array([3.0, 5.0, 7.0])
        title = "Test plot for linear data"
        # Instead of an assert statement, the test returns matplotlib figure.
        return get_plot_for_best_fit_line(slope, intercept, x_array, y_array, title)

    @pytest.mark.mpl_image_compare
    def test_plot_for_almost_linear_data(self):
        slope = -2.0
        intercept = 10.0
        x_array = np.array([1.0, 2.0, 3.0])
        y_array = np.array([8.0, 6.0, 5.0])
        title = "Test plot for almost linear data"
        # Instead of an assert statement, the test returns matplotlib figure.
        return get_plot_for_best_fit_line(slope, intercept, x_array, y_array, title)



Overwriting /content/mypackages/uregress/tests/visualization/test_plots.py


In [ ]:
import os
os.chdir("/content/mypackages/uregress")
!pip install -e .

Obtaining file:///content/mypackages/uregress
  Found existing installation: uregress 0.1.0
    Can't uninstall 'uregress'. No files were found to uninstall.
  Running setup.py develop for uregress


In [ ]:
import uregress
import pytest

In [ ]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items                                                             

tests/data/test_preprocessing_helpers.py ..............                  [ 53%]
tests/features/test_as_numpy.py ..                                       [ 61%]
tests/models/test_train.py ........                                      [ 92%]
tests/visualization/test_plots.py ..                                     [100%]

============================== 26 passed in 1.14s ==============================


In [ ]:
# Run pytest only in one particular file.

!pytest /content/mypackages/uregress/tests/data/test_preprocessing_helpers.py

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 14 items                                                             

tests/data/test_preprocessing_helpers.py ..............                  [100%]

============================== 14 passed in 1.06s ==============================


In [ ]:
# pytest using Node ID to test a particular test class
!pytest /content/mypackages/uregress/tests/data/test_preprocessing_helpers.py::TestRowToList

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 7 items                                                              

tests/data/test_preprocessing_helpers.py .......                         [100%]

============================== 7 passed in 1.06s ===============================


In [ ]:
# pytest using Node ID to test a particular test function
!pytest /content/mypackages/uregress/tests/data/test_preprocessing_helpers.py::TestRowToList::test_on_one_tab_with_missing_value

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 1 item                                                               

tests/data/test_preprocessing_helpers.py .                               [100%]

============================== 1 passed in 1.06s ===============================


In [ ]:
# Run pytest using -k flag
!pytest -k "TestSplit"

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 23 deselected / 3 selected                                

tests/models/test_train.py ...                                           [100%]

======================= 3 passed, 23 deselected in 1.09s =======================


In [ ]:
# Run pytest using -k flag with exception
!pytest -k "TestSplit and not test_on_one_row"


============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 24 deselected / 2 selected                                

tests/models/test_train.py ..                                            [100%]

======================= 2 passed, 24 deselected in 1.09s =======================


In [ ]:
# Changing this on purpose to see the failed pytest
%%file /content/mypackages/uregress/uregress/models/train.py
import numpy as np
from scipy.stats import linregress


def split_into_training_and_testing_sets(data_array):
    dim = data_array.ndim
    if dim != 2:
        raise ValueError("Argument data_array must be two dimensional. Got {0} dimensional array instead!".format(dim))
    num_rows = data_array.shape[0]
    if num_rows < 2:
        raise ValueError("Argument data_array must have at least 2 rows, it actually has just {0}".format(num_rows))
    num_training = int(0.2 * data_array.shape[0])
    permuted_indices = np.random.permutation(data_array.shape[0])
    return data_array[permuted_indices[:num_training], :], data_array[permuted_indices[num_training:], :]


def model_test(testing_set, slope, intercept):
    dim = testing_set.ndim
    if dim != 2:
        raise ValueError("Argument testing_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_cols = testing_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument testing_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    actual_price = testing_set[:, 1]
    predicted_price = slope*testing_set[:, 0] + intercept
    residual_sum_of_squares = np.sum(np.square(predicted_price - actual_price)) / testing_set.shape[0]
    return 1 - residual_sum_of_squares / np.var(actual_price)


def train_model(training_set):
    dim = training_set.ndim
    if dim != 2:
        raise ValueError("Argument training_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_rows = training_set.shape[0]
    if num_rows < 2:
        raise ValueError("Argument training_set must have at least 2 rows for linear regression to work, "
                         "it actually has just {0}".format(num_rows)
                         )
    num_cols = training_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument training_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    slope, intercept, _, _, _ = linregress(training_set[:, 0], training_set[:, 1])
    return slope, intercept


Overwriting /content/mypackages/uregress/uregress/models/train.py


In [ ]:
# This should generate AssertionError
!pytest /content/mypackages/uregress/tests/models/test_train.py::TestSplitIntoTrainingAndTestingSets

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 3 items                                                              

tests/models/test_train.py F..                                           [100%]

=================================== FAILURES ===================================
_____________ TestSplitIntoTrainingAndTestingSets.test_on_six_rows _____________

self = <tests.models.test_train.TestSplitIntoTrainingAndTestingSets object at 0x7fd319a3b850>

    def test_on_six_rows(self):
        test_argument = np.array([[2081.0, 314942.0],
                               [1059.0, 186606.0],
                               [1148.0, 206186.0],
                               [1506.0, 248419.0],
                               [1210.0, 214114.0],
                  

In [ ]:
# Changing back to the correct version
%%file /content/mypackages/uregress/uregress/models/train.py
import numpy as np
from scipy.stats import linregress


def split_into_training_and_testing_sets(data_array):
    dim = data_array.ndim
    if dim != 2:
        raise ValueError("Argument data_array must be two dimensional. Got {0} dimensional array instead!".format(dim))
    num_rows = data_array.shape[0]
    if num_rows < 2:
        raise ValueError("Argument data_array must have at least 2 rows, it actually has just {0}".format(num_rows))
    num_training = int(0.75 * data_array.shape[0])
    permuted_indices = np.random.permutation(data_array.shape[0])
    return data_array[permuted_indices[:num_training], :], data_array[permuted_indices[num_training:], :]


def model_test(testing_set, slope, intercept):
    dim = testing_set.ndim
    if dim != 2:
        raise ValueError("Argument testing_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_cols = testing_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument testing_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    actual_price = testing_set[:, 1]
    predicted_price = slope*testing_set[:, 0] + intercept
    residual_sum_of_squares = np.sum(np.square(predicted_price - actual_price)) / testing_set.shape[0]
    return 1 - residual_sum_of_squares / np.var(actual_price)


def train_model(training_set):
    dim = training_set.ndim
    if dim != 2:
        raise ValueError("Argument training_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_rows = training_set.shape[0]
    if num_rows < 2:
        raise ValueError("Argument training_set must have at least 2 rows for linear regression to work, "
                         "it actually has just {0}".format(num_rows)
                         )
    num_cols = training_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument training_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    slope, intercept, _, _, _ = linregress(training_set[:, 0], training_set[:, 1])
    return slope, intercept


Overwriting /content/mypackages/uregress/uregress/models/train.py


In [ ]:
# Run pytest again to see that the bug has been fixed.
!pytest /content/mypackages/uregress/tests/models/test_train.py::TestSplitIntoTrainingAndTestingSets

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 3 items                                                              

tests/models/test_train.py ...                                           [100%]

============================== 3 passed in 1.03s ===============================


A test class ```TestModelTest``` has been created within the test module ```models/test_train.py```. In the test class, there are two unit tests called ```test_on_linear_data()``` and ```test_on_one_dimensional_array()```. But the function ```model_test()``` has not been implemented yet.

In [1]:
# Commenting out mmodel_test on purpose

%%file /content/mypackages/uregress/uregress/models/train.py

import numpy as np
from scipy.stats import linregress


def split_into_training_and_testing_sets(data_array):
    dim = data_array.ndim
    if dim != 2:
        raise ValueError("Argument data_array must be two dimensional. Got {0} dimensional array instead!".format(dim))
    num_rows = data_array.shape[0]
    if num_rows < 2:
        raise ValueError("Argument data_array must have at least 2 rows, it actually has just {0}".format(num_rows))
    num_training = int(0.75 * data_array.shape[0])
    permuted_indices = np.random.permutation(data_array.shape[0])
    return data_array[permuted_indices[:num_training], :], data_array[permuted_indices[num_training:], :]


# def model_test(testing_set, slope, intercept):
#     dim = testing_set.ndim
#     if dim != 2:
#         raise ValueError("Argument testing_set must be two dimensional. Got {0} dimensional array instead!".format(
#                             dim
#                             )
#                          )
#     num_cols = testing_set.shape[1]
#     if num_cols != 2:
#         raise ValueError("Argument testing_set must have 2 columns for univariate linear regression. "
#                          "It actually has {0} columns".format(num_cols)
#                          )
#     actual_price = testing_set[:, 1]
#     predicted_price = slope*testing_set[:, 0] + intercept
#     residual_sum_of_squares = np.sum(np.square(predicted_price - actual_price)) / testing_set.shape[0]
#     return 1 - residual_sum_of_squares / np.var(actual_price)


def train_model(training_set):
    dim = training_set.ndim
    if dim != 2:
        raise ValueError("Argument training_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_rows = training_set.shape[0]
    if num_rows < 2:
        raise ValueError("Argument training_set must have at least 2 rows for linear regression to work, "
                         "it actually has just {0}".format(num_rows)
                         )
    num_cols = training_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument training_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    slope, intercept, _, _, _ = linregress(training_set[:, 0], training_set[:, 1])
    return slope, intercept

Overwriting /content/mypackages/uregress/uregress/models/train.py


In [6]:
!pytest /content/mypackages/uregress/tests/models/test_train.py::TestModelTest

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 0 items / 1 error                                                    

==================================== ERRORS ====================================
_________________ ERROR collecting tests/models/test_train.py __________________
ImportError while importing test module '/content/mypackages/uregress/tests/models/test_train.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/usr/lib/python3.7/importlib/__init__.py:127: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
mypackages/uregress/tests/models/test_train.py:5: in <module>
    from uregress.models.train import split_into_training_and_testing_sets, train_model, model_test
E   ImportError: c

In [56]:
%%file /content/mypackages/uregress/tests/models/test_train.py
from math import cos, pi, sin
import numpy as np
import pytest

from uregress.models.train import split_into_training_and_testing_sets, train_model, model_test


class TestSplitIntoTrainingAndTestingSets(object):
    def test_on_six_rows(self):
        test_argument = np.array([[2081.0, 314942.0],
                               [1059.0, 186606.0],
                               [1148.0, 206186.0],
                               [1506.0, 248419.0],
                               [1210.0, 214114.0],
                               [1697.0, 277794.0],
                               ]
                              )
        expected_length_training_set = 4
        expected_length_testing_set = 2
        actual = split_into_training_and_testing_sets(test_argument)
        assert actual[0].shape[0] == expected_length_training_set, \
               "The actual number of rows in the training array is not 4"
        assert actual[1].shape[0] == expected_length_testing_set, \
               "The actual number of rows in the testing array is not 2"

    def test_on_one_row(self):
        test_argument = np.array([[1382.0, 390167.0]])
        with pytest.raises(ValueError) as exc_info:
            split_into_training_and_testing_sets(test_argument)
        expected_error_msg = "Argument data_array must have at least 2 rows, it actually has just 1"
        assert exc_info.match(expected_error_msg)

    def test_on_one_dimensional_array(self):
        test_argument = np.array([1382.0, 390167.0])
        with pytest.raises(ValueError) as exc_info:
            split_into_training_and_testing_sets(test_argument)
        expected_error_msg = "Argument data_array must be two dimensional. Got 1 dimensional array instead!"
        assert exc_info.match(expected_error_msg)


class TestTrainModel(object):
    def test_on_linear_data(self):
        test_argument = np.array([[1.0, 3.0], [2.0, 5.0], [3.0, 7.0]])
        expected_slope = 2.0
        expected_intercept = 1.0
        actual_slope, actual_intercept = train_model(test_argument)
        slope_message = ("train_model({0}) should return slope {1}, "
                         "but it actually returned slope {2}".format(test_argument, expected_slope, actual_slope)
                         )
        intercept_message = ("train_model({0}) should return intercept {1}, "
                             "but it actually returned intercept {2}".format(test_argument,
                                                                             expected_intercept,
                                                                             actual_intercept
                                                                             )
                             )
        assert actual_slope == pytest.approx(expected_slope), slope_message
        assert actual_intercept == pytest.approx(expected_intercept), intercept_message

    def test_on_positively_correlated_data(self):
        test_argument = np.array([[1.0, 4.0], [2.0, 4.0],
                                  [3.0, 9.0], [4.0, 10.0],
                                  [5.0, 7.0], [6.0, 13.0],
                                  ]
                                 )
        actual_slope, actual_intercept = train_model(test_argument)
        assert actual_slope > 0, "Expected slope: > 0, Actual slope: {0}".format(actual_slope)


class TestModelTest(object):
    def test_on_perfect_fit(self):
        test_argument = np.array([[1.0, 3.0], [2.0, 5.0], [3.0, 7.0]])
        expected = 1.0
        actual = model_test(test_argument, 2.0, 1.0)
        message = "model_test({0}) should return {1}, but it actually returned {2}".format(test_argument,
                                                                                           expected,
                                                                                           actual
                                                                                           )
        assert actual == pytest.approx(expected), message

    def test_on_circular_data(self):
        theta = pi / 4.0
        test_argument = np.array([[0.0, 1.0],
                                  [cos(theta), sin(theta)],
                                  [1.0, 0.0],
                                  [cos(3 * theta), sin(3 * theta)],
                                  [0.0, -1.0],
                                  [cos(5 * theta), sin(5 * theta)],
                                  [-1.0, 0.0],
                                  [cos(7 * theta), sin(7 * theta)]
                                  ]
                                 )
        actual = model_test(test_argument, 0.0, 0.0)
        assert actual == pytest.approx(0.0)

    def test_on_one_dimensional_array(self):
        test_argument = np.array([1.0, 2.0, 3.0, 4.0])
        with pytest.raises(ValueError) as exc_info:
            model_test(test_argument, 1.0, 1.0)
        expected_error_msg = "Argument testing_set must be two dimensional. Got 1 dimensional array instead!"
        assert exc_info.match(expected_error_msg)




Overwriting /content/mypackages/uregress/tests/models/test_train.py


In [8]:
!pytest /content/mypackages/uregress/tests/models/test_train.py::TestModelTest

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 0 items / 1 error                                                    

==================================== ERRORS ====================================
_________________ ERROR collecting tests/models/test_train.py __________________
ImportError while importing test module '/content/mypackages/uregress/tests/models/test_train.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/usr/lib/python3.7/importlib/__init__.py:127: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
mypackages/uregress/tests/models/test_train.py:5: in <module>
    from uregress.models.train import split_into_training_and_testing_sets, train_model, model_test
E   ImportError: c

# ทำไม xfail ใช้ไม่ได้ ?



In [9]:
# Change back to normal code

%%file /content/mypackages/uregress/uregress/models/train.py

import numpy as np
from scipy.stats import linregress


def split_into_training_and_testing_sets(data_array):
    dim = data_array.ndim
    if dim != 2:
        raise ValueError("Argument data_array must be two dimensional. Got {0} dimensional array instead!".format(dim))
    num_rows = data_array.shape[0]
    if num_rows < 2:
        raise ValueError("Argument data_array must have at least 2 rows, it actually has just {0}".format(num_rows))
    num_training = int(0.75 * data_array.shape[0])
    permuted_indices = np.random.permutation(data_array.shape[0])
    return data_array[permuted_indices[:num_training], :], data_array[permuted_indices[num_training:], :]


def model_test(testing_set, slope, intercept):
    dim = testing_set.ndim
    if dim != 2:
        raise ValueError("Argument testing_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_cols = testing_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument testing_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    actual_price = testing_set[:, 1]
    predicted_price = slope*testing_set[:, 0] + intercept
    residual_sum_of_squares = np.sum(np.square(predicted_price - actual_price)) / testing_set.shape[0]
    return 1 - residual_sum_of_squares / np.var(actual_price)


def train_model(training_set):
    dim = training_set.ndim
    if dim != 2:
        raise ValueError("Argument training_set must be two dimensional. Got {0} dimensional array instead!".format(
                            dim
                            )
                         )
    num_rows = training_set.shape[0]
    if num_rows < 2:
        raise ValueError("Argument training_set must have at least 2 rows for linear regression to work, "
                         "it actually has just {0}".format(num_rows)
                         )
    num_cols = training_set.shape[1]
    if num_cols != 2:
        raise ValueError("Argument training_set must have 2 columns for univariate linear regression. "
                         "It actually has {0} columns".format(num_cols)
                         )
    slope, intercept, _, _, _ = linregress(training_set[:, 0], training_set[:, 1])
    return slope, intercept

Overwriting /content/mypackages/uregress/uregress/models/train.py


In [12]:
!pytest /content/mypackages/uregress/tests/models/test_train.py::TestModelTest

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 3 items                                                              

mypackages/uregress/tests/models/test_train.py ...                       [100%]

============================== 3 passed in 1.06s ===============================


```range()``` has been deliberately replaced with the obsolete ```xrange()``` in the function /content/mypackages/uregress/uregress/features/as_numpy.py >>> get_data_as_numpy_array

Run the tests in the test class ```TestGetDataAsNumpyArray``` in the IPython console. What is the outcome?

In [14]:
!pytest -k "TestGetDataAsNumpyArray"

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content
plugins: mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 24 deselected / 2 selected                                

mypackages/uregress/tests/features/test_as_numpy.py FF                   [100%]

=================================== FAILURES ===================================
__________________ TestGetDataAsNumpyArray.test_on_clean_file __________________

self = <tests.features.test_as_numpy.TestGetDataAsNumpyArray object at 0x7f1cd83b6950>
clean_data_file = 'clean.txt'

    def test_on_clean_file(self, clean_data_file):
        expected = np.array([[201.0, 305671.0], [7892.0, 298140.0], [501.0, 738293.0]])
>       actual = get_data_as_numpy_array(clean_data_file, 2)

mypackages/uregress/tests/features/test_as_numpy.py:28: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

ลองใส่ skipif ที่ไฟล์ /content/mypackages/uregress/tests/features/test_as_numpy.py

# สรุป xfail และ skipif ใช้ไม่ได้ทั้งคู่

In [32]:
%%file /content/mypackages/uregress/.travis.yml
language: python
python: 
  - "3.6"
install:
  - pip install -e .
  - pip install  pytest-cov codecov
script:
  - pytest --cov=uregress tests
after_success:
  codecov


Overwriting /content/mypackages/uregress/.travis.yml


In [33]:
!pip install  pytest-cov codecov

     |████████████████████████████████| 245kB 11.9MB/s 
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.5 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1


In [35]:
!pytest --cov=/content/mypackages/uregress/tests

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content
plugins: cov-2.12.0, mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items                                                             

mypackages/uregress/tests/data/test_preprocessing_helpers.py ........... [ 42%]
...                                                                      [ 53%]
mypackages/uregress/tests/features/test_as_numpy.py ..                   [ 61%]
mypackages/uregress/tests/models/test_train.py ........                  [ 92%]
mypackages/uregress/tests/visualization/test_plots.py ..                 [100%]

---------- coverage: platform linux, python 3.7.10-final-0 -----------
Name                                                           Stmts   Miss  Cover
----------------------------------------------------------------------------------
mypackages/uregre

In [37]:
import codecov
!codecov


      _____          _
     / ____|        | |
    | |     ___   __| | ___  ___ _____   __
    | |    / _ \ / _  |/ _ \/ __/ _ \ \ / /
    | |___| (_) | (_| |  __/ (_| (_) \ V /
     \_____\___/ \____|\___|\___\___/ \_/
                                    v2.1.11

==> Detecting CI provider
fatal: not a git repository (or any of the parent directories): .git
    Error running `['git', 'rev-parse', '--abbrev-ref', 'HEAD']`: returncode=128, output=b''
    Error running `['hg', 'branch']`: returncode=None, output=[Errno 2] No such file or directory: 'hg': 'hg'
  -> Got branch from git/hg
fatal: not a git repository (or any of the parent directories): .git
    Error running `['git', 'rev-parse', 'HEAD']`: returncode=128, output=b''
    Error running `['hg', 'log', '-r', '.', '-T', '{node}']`: returncode=None, output=[Errno 2] No such file or directory: 'hg': 'hg'
  -> Got sha from git/hg
==> Preparing upload
Error: Commit sha is missing. Please specify via --commit=:sha
Tip: See all exampl

# ควรจะต้องทำบน github แบบเป็นเรื่องเป็นราว ไว้ทีหลัง

In [ ]:
%%file /content/mypackages/uregress/uregress/data/preprocessing_helpers.py

def convert_to_int(integer_string_with_commas):
    comma_separated_parts = integer_string_with_commas.split(",")
    for i in range(len(comma_separated_parts)):
        if len(comma_separated_parts[i]) > 3:
            return None
        if i != 0 and len(comma_separated_parts[i]) != 3:
            return None
    integer_string_without_commas = "".join(comma_separated_parts)
    try:
        return int(integer_string_without_commas)
    except ValueError:
        return None


def row_to_list(row):
    row = row.rstrip("\n")
    separated_entries = row.split("\t")
    if len(separated_entries) == 2 and "" not in separated_entries:
        return separated_entries
    return None


def preprocess(raw_data_file_path, clean_data_file_path):
    with open(raw_data_file_path, "r") as input_file:
        rows = input_file.readlines()
    with open(clean_data_file_path, "w") as output_file:
        for row in rows:
            row_as_list = row_to_list(row)
            if row_as_list is None:
                continue
            area = convert_to_int(row_as_list[0])
            price = convert_to_int(row_as_list[1])
            if area is None or price is None:
                continue
            output_file.write("{0}\t{1}\n".format(area, price))

# Use a fixture for a clean data file
# Write a fixture for an empty data file
# Fixture chaining using tmpdir

## ดูตัวอย่างไฟล์ได้ที่
/content/mypackages/uregress/tests/features/test_as_numpy.py

# Program a bug-free dependency
# Mock a dependency


/content/mypackages/uregress/tests/data/test_preprocessing_helpers.py

# Testing on linear data
# Testing on circular data
/content/mypackages/uregress/data/circula_data.png

/content/mypackages/uregress/tests/models/test_train.py

# Generating the Baseline Images
```/content/mypackages/uregress/uregress/visualization/plots.py```

```/content/mypackages/uregress/tests/visualization/test_plots.py```

## Use the command (in case the directories have not yet been built.)

```!pytest -k "test_plot_for_linear_data" --mpl-generate-path /content/mypackages/uregress/tests/visualization/baseline```

```!pytest -k "test_plot_for_almost_linear_data" --mpl-generate-path /content/mypackages/uregress/tests/visualization/baseline```

## The next time we run the test, just type

```!pytest -k "test_plot_for_linear_data" --mpl```

```!pytest -k "test_plot_for_almost_linear_data" --mpl```

### ถ้าภาพไม่เหมือนกันจะถูกนำไปเก็บไว้ที่ temporary folder

# Generate the baseline image EXERCISE



*   Execute the command required to create the baseline image for this test only. The baseline folder should be in ```uregress/tests/visualization```



In [65]:
### Generate wrong axis label on purpose

%%file /content/mypackages/uregress/uregress/visualization/plots.py
import matplotlib.pyplot as plt
import numpy as np


def get_plot_for_best_fit_line(slope, intercept, x_array, y_array, title):
    x_array_ndim = x_array.ndim
    if not x_array_ndim == 1:
        raise ValueError("Argument x_array should be 1 dimensional. "
                         "It actually is {0} dimensional".format(x_array_ndim)
                         )
    y_array_ndim = y_array.ndim
    if not y_array_ndim == 1:
        raise ValueError("Argument y_array should be 1 dimensional. "
                         "It actually is {0} dimensional".format(y_array_ndim)
                         )
    x_array_length = x_array.shape[0]
    y_array_length = y_array.shape[0]
    if x_array_length != y_array_length:
        raise RuntimeError("Arguments x_array and y_array should have same length. "
                           "But x_array has length {0} and y_array has length {1}".format(x_array_length,
                                                                                          y_array_length
                                                                                          )
                           )
    fig, ax = plt.subplots()
    ax.plot(x_array, y_array, ".")
    ax.plot([0, np.max(x_array)], [intercept, slope * np.max(x_array) + intercept], "-")
    ax.set(xlabel="price (dollars)", ylabel="area (square feet)", title=title)
    return fig


Overwriting /content/mypackages/uregress/uregress/visualization/plots.py


In [64]:
!pytest --mpl-generate-path /content/mypackages/uregress/tests/visualization/baseline -k "test_plot_for_almost_linear_data"
!pytest --mpl-generate-path /content/mypackages/uregress/tests/visualization/baseline -k "test_plot_for_linear_data"

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content
plugins: cov-2.12.0, mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 25 deselected / 1 selected                                

mypackages/uregress/tests/visualization/test_plots.py s                  [100%]

====================== 1 skipped, 25 deselected in 1.21s =======================
============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content/mypackages/uregress
plugins: cov-2.12.0, mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 25 deselected / 1 selected                                

mypackages/uregress/tests/visualization/test_plots.py s                  [100%]

====================== 1 skipped, 25 deselected

In [66]:
!pytest -k "TestGetPlotForBestFitLine" --mpl

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content
plugins: cov-2.12.0, mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 24 deselected / 2 selected                                

mypackages/uregress/tests/visualization/test_plots.py FF                 [100%]

=================================== FAILURES ===================================
_____________ TestGetPlotForBestFitLine.test_plot_for_linear_data ______________
Error: Image files did not match.
  RMS Value: 11.31222568728188
  Expected:  
    /tmp/tmpbnzcfp0g/tests.visualization.test_plots.test_plot_for_linear_data/baseline.png
  Actual:    
    /tmp/tmpbnzcfp0g/tests.visualization.test_plots.test_plot_for_linear_data/result.png
  Difference:
    /tmp/tmpbnzcfp0g/tests.visualization.test_plots.test_plot_for_linear_data/result-failed-diff.png
  Tolerance: 
    2
____

# The black areas are where the actual image and the baseline matches. The white areas are where they don't match.

### Where are they?  I couldn't locate the difference file on colab ?

What happens after correcting the plot function?

In [68]:
!pytest -k "TestGetPlotForBestFitLine" --mpl

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Matplotlib: 3.2.2
Freetype: 2.6.1
rootdir: /content
plugins: cov-2.12.0, mock-3.6.1, mpl-0.12, typeguard-2.7.1
collected 26 items / 24 deselected / 2 selected                                

mypackages/uregress/tests/visualization/test_plots.py ..                 [100%]

======================= 2 passed, 24 deselected in 1.66s =======================
